# Classifying emotions in tweets using Watson NLP

This notebook demonstrates how to classify emotions in tweets using Watson NLP python library

The dataset contains over three thousand quotations from movie dialogues. The labeled emotions in the dataset are "anger", "sadness", "fear", "joy", and "disgust". Because some quotations have multiple sentences, they can be regarded as documents. You can download the dataset from [Github Repo](https://github.ibm.com/hcbt/Watson-NLP/blob/main/Emotion-Classification/movieDialog_train.csv)

### What you'll learn in this notebook
Watson NLP offers so-called blocks for various NLP tasks. This notebook shows:

- **Syntax analysis** with the _Syntax block_ for English (`syntax_izumo_en_stock`). This block performs tokenization, lemmatization, parts of speech tagging, and dependency parsing on raw input documents so that custom models can properly classify documents.
- **Emotion classification** with the _Ensemble emotion workflow_ (`ensemble_classification-wf_en_emotion-stock`) and the _Aggregated emotion workflow_ (`aggregated_classification-wf_en_emotion-stock`). These model workflow classify text into five emotions: "sadness", "joy", "anger", "fear", "disgust".

## Table of Contents

1. [Before you start](#beforeYouStart)
1. [Train Test Split](#trainTestSplit)
1. [Running pretrained out-of-the-box models](#pretrainedOOTB)

    1. [Ensemble emotion model](#ensemble)
    1. [Aggregated emotion workflow](#aggregated)
    
1. [Training models](#training)

    1. [Pre-processing](#preProcessing)
    1. [TF-IDF](#tfidf)
    1. [Embeddings](#embeddings)
    1. [CNN](#cnn)
    
1. [Testing](#testing)
1. [Exploring document level token level predictions](#words)

<a id="beforeYouStart"></a>
## 1. Before you start

<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.8 + Watson NLP XS_ environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

<span style="color:blueviolet">Begin by importing and initializing some helper libs that are used throughout the notebook.</span>


In [2]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import watson_nlp
import watson_nlp.data_model as dm

from sklearn.model_selection import train_test_split

from watson_core.toolkit import fileio
from watson_core.toolkit.quality_evaluation import QualityEvaluator, EvalTypes

from watson_nlp.blocks.classification.bert import BERT
from watson_nlp.blocks.classification.cnn import CNN
from watson_core.data_model.streams.resolver import DataStreamResolver
from watson_core.data_model.streams.resolver import DataStream
from watson_nlp.blocks.classification.svm import SVM
from watson_nlp.blocks.vectorization.tfidf import TFIDF

In [4]:
pd.set_option('display.max_colwidth', 0)

<span style="color:maroon">Printing either `block_models` or `workflow_models` will display a list of pretrained models available in the current version of Watson NLP</span>

In [5]:
block_models = watson_nlp.get_models().get_alias_models()
workflow_models = watson_nlp.get_workflows().get_alias_models()

<a id="trainTestSplit"></a>
## 2. Train Test Split

<div class="alert alert-block alert-info">
<b>Tip:</b> If you want to carry out emotion analysis on any other dataset, you should first upload the dataset into the project and then update the name of the file in the next cell</div>
<br>

<span style="color:blue">The data will be split into an 80/20 train-test split using sklearn and then exported into JSON format for the Watson NLP models to consume. Additionally, the column headers will be renamed to the expected `text` and `labels` names, with the labels having type list.</span>

In [6]:
file = project.get_file('movieDialog_train.csv')
df = pd.read_csv(file)

In [7]:
def convertToList(x):
    return [x]

In [8]:
df['label'] = df['label'].apply(convertToList)
df = df.rename(columns={'label':'labels'})

In [9]:
df_train, df_test = train_test_split(df, test_size=0.2)

In [10]:
df_train.to_json('movieDialog_train.json', orient='records')
df_test.to_json('movieDialog_test.json', orient='records')

In [11]:
df_train

,text,labels,source
1145,Only one guys checked in?,[disgust],MovieDialog
3307,"That's what I mean - mysterious. Mr. Conway, I don't like that man. He's too vague.",[disgust],MovieDialog
357,You don't mind that I'm not coming tonight?,[fear],MovieDialog
1272,What do I think? You mean about the possibility of your becoming a monster in two days or about visits from dead friends?,[anger],MovieDialog
2356,I was referring to myself. I thought we might have a picnic tomorrow - it might be a nice change after the Wild West party tonight. Invite everybody to go to the Everglades -,[happiness],MovieDialog
...,...,...,...
1614,What would you say if I told you the toilet just blew up in my face.,[anger],MovieDialog
2327,"--ohGodplease -- don't kill me -- don't kill me -- you're one of them, I know it --",[fear],MovieDialog
3532,"If we run into Billy first, let me try and talk him in.",[neutral],MovieDialog
1504,If he's gonna fuck me up the ass!,[disgust],MovieDialog


<a id="pretrainedOOTB"></a>
## 3. Running pretrained out-of-the-box models
<span style="color:blue">Watson NLP has two pretrained/prebuilt emotion classificaiton models using the workflow system. 
The following examples will use *\"Such a sweet boy. But after much thought and careful consideration, I've decided that the ruler for the next ten thousand years is going to have to be... me.\"* as a single input test with the expected label to be *\"joy\"*.</span>

<a id="ensemble"></a>
### Ensemble emotion model
<br>
<span style="color:blue">Ensemble emotion model will perform document emotion classification. [Docs](https://pages.github.ibm.com/ai-foundation/watson-nlp-documentation/workflow_emotion_classification.html)</span>

In [13]:
# Load the Emotion workflow model for English
ensemble_emotion_model = watson_nlp.load(watson_nlp.download('ensemble_classification-wf_en_emotion-stock'))

<span style="color:blueviolet">The result of the Ensemble model returns a list of dictionaries that indicate each label and its confidence score. The highest confidence score deems the label of prediction.</span>

In [14]:
# Run the Emotion model on a single document
ensemble_emotion_result = ensemble_emotion_model.run("Such a sweet boy. But after much thought and careful consideration, I've decided that the ruler for the next ten thousand years is going to have to be... me. ")
print(ensemble_emotion_result)

{
  "classes": [
    {
      "class_name": "joy",
      "confidence": 0.520251523364674
    },
    {
      "class_name": "sadness",
      "confidence": 0.13437655658432934
    },
    {
      "class_name": "anger",
      "confidence": 0.030973352040305283
    },
    {
      "class_name": "fear",
      "confidence": 0.023922530189972323
    },
    {
      "class_name": "disgust",
      "confidence": 0.01345159203717203
    }
  ],
  "producer_id": {
    "name": "Voting based Ensemble",
    "version": "0.0.1"
  }
}


<span style="color:blueviolet">The Ensemble model also has the `model.evaluate_quality()` function, which allows the evaluation of an entire test set. The result returns a dictionary of dictionaries that map each label to its confusion matrix, precision and recall, and f1 score.</span>

In [15]:
test_data_file = "movieDialog_test.json"

quality_report = ensemble_emotion_model.evaluate_quality(test_data_file)
print(json.dumps(quality_report, indent=4))

{
    "per_class_confusion_matrix": {
        "anger": {
            "true_positive": 155,
            "true_negative": 0,
            "false_positive": 61,
            "false_negative": 126,
            "precision": 0.7175925925925926,
            "recall": 0.5516014234875445,
            "f1": 0.6237424547283702
        },
        "fear": {
            "true_positive": 7,
            "true_negative": 0,
            "false_positive": 84,
            "false_negative": 31,
            "precision": 0.07692307692307693,
            "recall": 0.18421052631578946,
            "f1": 0.10852713178294575
        },
        "sadness": {
            "true_positive": 70,
            "true_negative": 0,
            "false_positive": 174,
            "false_negative": 41,
            "precision": 0.28688524590163933,
            "recall": 0.6306306306306306,
            "f1": 0.39436619718309857
        },
        "happiness": {
            "true_positive": 0,
            "true_negative": 0,
      

Based on the micro precision, recall, and f1 score, this Ensemble model did not perform well on our test data. However hyperparameter tuning of the out-of-the-box model may improve its performance.
<br><br>
*The micro metrics are taken here instead of the macro metrics because of the class imbalance in the data.

<a id="aggregated"></a>
### Aggregated emotion workflow
Aggregated emotion model has the capability of specifying target words in addition to document emotion classification. [Docs](https://pages.github.ibm.com/ai-foundation/watson-nlp-documentation/workflow_emotion_aggregated.html)

In [16]:
# Load the Emotion workflow model for English
aggregated_emotion_model = watson_nlp.load(watson_nlp.download('aggregated_classification-wf_en_emotion-stock'))

<span style="color:blue">By defining a target span of indices with the `target_mentions` parameter, the Aggregated model can predict emotion on exact segments of the input. </span>
<br><br>
<span style="color:blueviolet">Similarly to the Ensemble model, the Aggregated model returns a list of dictionaries that indicate each label and its confidence score. As before, the highest confidence score deems the label of prediction. 
The first dictionary in the result is the prediction of the overall input; the subsequent dictionaries with a \"target\" key is the prediction of each targeted text span.</span>

In [18]:
# span targets a section of the document given indices
target_mentions = dm.TargetMentionsPrediction([dm.TargetMentions([(7, 16)]), dm.TargetMentions([(28, 66)])])

aggregated_emotion_result_span = aggregated_emotion_model.run("Such a sweet boy. But after much thought and careful consideration, I've decided that the ruler for the next ten thousand years is going to have to be... me. ", 
                                   target_mentions=target_mentions)

print(aggregated_emotion_result_span)

{
  "emotion_predictions": [
    {
      "emotion": {
        "anger": 0.014531953677202513,
        "disgust": 0.015678834170103073,
        "fear": 0.004446345653956418,
        "joy": 0.9617643175703107,
        "sadness": 0.04595572027293119
      },
      "target": "sweet boy",
      "emotion_mentions": [
        {
          "span": {
            "begin": 0,
            "end": 17,
            "text": "Such a sweet boy."
          },
          "emotion": {
            "anger": 0.014531953677202513,
            "disgust": 0.015678834170103073,
            "fear": 0.004446345653956418,
            "joy": 0.9617643175703107,
            "sadness": 0.04595572027293119
          }
        }
      ]
    },
    {
      "emotion": {
        "anger": 0.16141723276990833,
        "disgust": 0.016278604327729256,
        "fear": 0.08282793606772569,
        "joy": 0.0715819208910971,
        "sadness": 0.2009543270775766
      },
      "target": "much thought and careful consideration",
     

<span style="color:blueviolet">By defining a target text phrase with the `target_phrases` parameter, the Aggregated model can predict emotion on exact words and phrases in the input. 
The result is just like the previous return, but this time with the \"target\" key having the targeted phrase.</span>

In [19]:
# text targets a section of the document given phrases
target_phrases = ['sweet boy', 'careful consideration']

aggregated_emotion_result_text = aggregated_emotion_model.run("Such a sweet boy. But after much thought and careful consideration, I've decided that the ruler for the next ten thousand years is going to have to be... me. ", 
                                   target_phrases=target_phrases)

print(aggregated_emotion_result_text)

{
  "emotion_predictions": [
    {
      "emotion": {
        "anger": 0.014531953677202513,
        "disgust": 0.015678834170103073,
        "fear": 0.004446345653956418,
        "joy": 0.9617643175703107,
        "sadness": 0.04595572027293119
      },
      "target": "sweet boy",
      "emotion_mentions": [
        {
          "span": {
            "begin": 0,
            "end": 17,
            "text": "Such a sweet boy."
          },
          "emotion": {
            "anger": 0.014531953677202513,
            "disgust": 0.015678834170103073,
            "fear": 0.004446345653956418,
            "joy": 0.9617643175703107,
            "sadness": 0.04595572027293119
          }
        }
      ]
    },
    {
      "emotion": {
        "anger": 0.16141723276990833,
        "disgust": 0.016278604327729256,
        "fear": 0.08282793606772569,
        "joy": 0.0715819208910971,
        "sadness": 0.2009543270775766
      },
      "target": "careful consideration",
      "emotion_mention

<a id="Training models"></a>
## 4. Training models

<span style="color:blue">There are three custom classification models in the follow sections. TF-IDF and Embeddings SVM models are trained and saved but not used in the testing section. They can be tested as a bonus. The CNN model is set up to be fine tuned and tested on the test split.</span>
<br>
<br>
<span style="color:blueviolet">The `model.save()` function will save the model as a directory populated with a config.yaml and an artifacts folder. The model will save in the current working directory where this notebook was launched, unless otherwise specified in the model name.</span>

<a id="preProcessing"></a>
### Pre-Processing

<span style="color:blue">The pre-processing step converts the training dataset into a a data stream for Watson NLP consumption. 
The training data is then run through a syntax model to perform tokenization and lemmatization.
The following three models will all use this syntax processed data as training data.</span>

In [12]:
training_data_file = "movieDialog_train.json"

# Create datastream from training data
data_stream_resolver = DataStreamResolver(target_stream_type=list, expected_keys={'text': str, 'labels': list})
training_data = data_stream_resolver.as_data_stream(training_data_file)

# Load a Syntax model
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))

# Create Syntax stream
text_stream, labels_stream = training_data[0], training_data[1]
syntax_stream = syntax_model.stream(text_stream)

<a id="tfidf"></a>
### Training with TF-IDF

In [20]:
# Train the TF-IDF vectorizer
tf_idf_model = TFIDF.train(syntax_stream)
tfidf_train_stream = tf_idf_model.stream(syntax_stream)
tfidf_svm_train_stream = watson_nlp.data_model.DataStream.zip(tfidf_train_stream, labels_stream)

# Train SVM using TF-IDF training stream
tfidf_classification_model = SVM.train(tfidf_svm_train_stream)

In [21]:
tfidf_classification_model.save('model_tfidf_emo_classification')

<a id="embeddings"></a>
### Training with Embeddings

In [22]:
use_embedding_model = watson_nlp.download_and_load('embedding_use_en_stock')
use_train_stream = use_embedding_model.stream(syntax_stream, doc_embed_style='raw_text')
# `raw_text`: run the universal sentence encoder over your text as one large chunk
# `ave_sent`: independently run the universal sentence encoder over each of your sentences and average the results to 
#             produce a document embedding
use_svm_train_stream = watson_nlp.data_model.DataStream.zip(use_train_stream, labels_stream)

# Train SVM using Universal Sentence Encoder (USE) training stream
embeddings_classification_model = SVM.train(use_svm_train_stream)

In [23]:
embeddings_classification_model.save('model_embeddings_emo_classification')

<a id="cnn"></a>
### Training with CNN
<span style="color:maroon">These hyperparameters can be used in the out-of-the-box to tune those models too.</span>

In [24]:
training_data_file = "movieDialog_train.json"

# Load glove embeddings
glove_embedding_model = watson_nlp.download_and_load('embedding_glove_en_stock')

# Train CNN
cnn_classification_model = CNN.train(DataStream.zip(syntax_stream, labels_stream), 
                                     embedding=glove_embedding_model.embedding, 
                                     batch_size=128, 
                                     filter_sizes=(1, 2, 2), 
                                     num_filters=256, 
                                     epochs=10, 
                                     random_seed=1001, 
                                     enable_tensorboard=False, 
                                     dropout_prob=0.5, 
                                     l2_reg_lambda=0.01, 
                                     verbose=1, 
                                     multi_label=False, )

Epoch 1/10
27/27 [==============================] - 26s 915ms/step - loss: 5.4816 - categorical_accuracy: 0.3486
Epoch 2/10
27/27 [==============================] - 24s 899ms/step - loss: 4.0135 - categorical_accuracy: 0.4577
Epoch 3/10
27/27 [==============================] - 24s 896ms/step - loss: 3.0717 - categorical_accuracy: 0.5092
Epoch 4/10
27/27 [==============================] - 24s 905ms/step - loss: 2.4441 - categorical_accuracy: 0.5368
Epoch 5/10
27/27 [==============================] - 24s 899ms/step - loss: 2.0339 - categorical_accuracy: 0.5778
Epoch 6/10
27/27 [==============================] - 24s 900ms/step - loss: 1.7635 - categorical_accuracy: 0.6063
Epoch 7/10
27/27 [==============================] - 25s 913ms/step - loss: 1.5709 - categorical_accuracy: 0.6208
Epoch 8/10
27/27 [==============================] - 25s 934ms/step - loss: 1.4303 - categorical_accuracy: 0.6316
Epoch 9/10
27/27 [==============================] - 25s 916ms/step - loss: 1.3355 - categorical_

In [25]:
cnn_classification_model.save('model_cnn_emo_classification')

<span style="color:blueviolet">By using `project.save_data()`, the model will be saved as an object into the Data Assets of the Watson Studio project.</span>

In [26]:
project.save_data('model_cnn_emo_classification', data=cnn_classification_model.as_file_like_object(), overwrite=True)

{'file_name': 'model_cnn_emo_classification',
 'message': 'File saved to project storage.',
 'bucket_name': 'watsoncore-donotdelete-pr-olkxvfa8bk0pb1',
 'asset_id': '0e8343b6-0009-4cca-903b-0aec5ff4e27c'}

<a id="Testing"></a>
## 5. Testing
<span style="color:blue">The first step in testing the model is to load in the previously saved classification model. Then, just like with the training data, the testing data will need to be processed by the syntax model for tokenization and lemmatization. Finally, the `model.evaluate_quality()` function will run predictions on the data.</span>

In [27]:
test_data_file = "movieDialog_test.json"

# Load a Syntax model
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))

# Load classification model
classification_model = watson_nlp.load('model_cnn_emo_classification')

# Setup pre-processing function
preprocess_func = lambda raw_doc: syntax_model.run_batch(raw_doc)

quality_report = classification_model.evaluate_quality(test_data_file, preprocess_func)

<span style="color:blueviolet">While this CNN classification model did out perform the out-of-the-box Ensemble model, it still has a medium-low micro precision, recall, and f1.
<br><br>
*The micro metrics are taken here instead of the macro metrics because of the class imbalance in the data.</span>

In [28]:
print(json.dumps(quality_report, indent=4))

{
    "per_class_confusion_matrix": {
        "anger": {
            "true_positive": 203,
            "true_negative": 0,
            "false_positive": 189,
            "false_negative": 78,
            "precision": 0.5178571428571429,
            "recall": 0.7224199288256228,
            "f1": 0.6032689450222882
        },
        "fear": {
            "true_positive": 0,
            "true_negative": 0,
            "false_positive": 0,
            "false_negative": 38,
            "precision": 0,
            "recall": 0.0,
            "f1": 0
        },
        "neutral": {
            "true_positive": 174,
            "true_negative": 0,
            "false_positive": 148,
            "false_negative": 99,
            "precision": 0.5403726708074534,
            "recall": 0.6373626373626373,
            "f1": 0.5848739495798319
        },
        "happiness": {
            "true_positive": 59,
            "true_negative": 0,
            "false_positive": 40,
            "false_negati

<span style="color:blueviolet">With the same single input, the CNN model predicts `happiness` with higher confidence than the out-of-the-box models.</span>

In [29]:
syntax_prediction = syntax_model.run("Such a sweet boy. But after much thought and careful consideration, I've decided that the ruler for the next ten thousand years is going to have to be... me. ")
classifier_result = classification_model.run(syntax_prediction)
print(classifier_result)

{
  "classes": [
    {
      "class_name": "happiness",
      "confidence": 0.7313699722290039
    },
    {
      "class_name": "neutral",
      "confidence": 0.0853412076830864
    },
    {
      "class_name": "sadness",
      "confidence": 0.08039671927690506
    },
    {
      "class_name": "anger",
      "confidence": 0.04342210665345192
    },
    {
      "class_name": "disgust",
      "confidence": 0.04208728298544884
    },
    {
      "class_name": "fear",
      "confidence": 0.017382703721523285
    }
  ],
  "producer_id": {
    "name": "CNN classifier",
    "version": "0.0.1"
  }
}


In [30]:
# Another option for saving the model
'''
import pickle
with open ('model_cnn_emo_classification.pkl', 'wb') as f:
    pickle.dump(classification_model.as_file_like_object(), f)
'''

"\nimport pickle\nwith open ('model_cnn_emo_classification.pkl', 'wb') as f:\n    pickle.dump(classification_model.as_file_like_object(), f)\n"

<a id="words"></a>
## 6. Exploring document level token level predictions
<span style="color:blue">The cells that follow are an exploration of the Aggregated model with regard to document level and token level predictions. The model returns will be constructed into dataframes to better view results.</span>

In [31]:
# set the emotion model as the aggregated out of the box model because of its target mention capability
emotion_model = aggregated_emotion_model

In [32]:
def extract_emotion(text):
    # run the emotion model on the result of the syntax analysis
    emotion_result = emotion_model.run(text, document_emotion=True)
    
    document_emotion = emotion_result.to_dict()['emotion_predictions'][0]['emotion']
    mention_emotion = [(sm['span']['text'], sm['emotion']) for sm in emotion_result.to_dict()['emotion_predictions'][0]['emotion_mentions']]
    return (document_emotion, mention_emotion)

# Helper method to create a new dataframe with the corresponding emotion
def create_emotion_dataframe(df):
    emotion = df['text'].apply(lambda text: extract_emotion(text))
    emotion_df = pd.DataFrame.from_records(emotion, columns=('Document emotion', 'Mention emotion'))
    return emotion_df

In [33]:
# this will take some time
emotion_df = create_emotion_dataframe(df)

<span style="color:blue">In the context of the dataset, each row is a different and unrelated movie dialog from the entire dataset (before train-test split).</span>

In [35]:
document_emotion_df = df[['text']].merge(pd.DataFrame(emotion_df['Document emotion'].values.tolist()), 
                                          how='left', 
                                          left_index=True, 
                                          right_index=True)
document_emotion_df

,text,anger,disgust,fear,joy,sadness
0,"No... sort of. Man, she could be cool, but all she does is get wrecked and do all the guys. She's blowin' them in the parking lot.",0.091589,0.101249,0.127507,0.179821,0.280981
1,Poker sounds great. When do you play?,0.070759,0.047147,0.089574,0.533756,0.179207
2,Washington! Always discussing the problems of Washington. Nobody ever thinks of the State--and my problems! I *will* tell Jim Taylor. It's high *time* I told him a thing or two!,0.222626,0.040641,0.185320,0.173301,0.250626
3,"You know, Julie, even if this works - which it probably won't - that stuff is tricky. You don't know what it'll do to his brain.",0.130315,0.030532,0.311909,0.043280,0.298986
4,"David, please be rational. Let's go to Dr. Hirsch.",0.048054,0.017707,0.472761,0.230007,0.088004
...,...,...,...,...,...,...
4294,"You can't have nothing, Jill. We need lunch. Mr. Pizza?",0.103425,0.030539,0.064858,0.386445,0.150501
4295,"You can't help me, man.",0.152057,0.050473,0.113592,0.105413,0.468593
4296,You don't know. I'm sorry. I was desperate. That's not me. I shot a cop. Can you imagine what they'll do to me when I got to prison?,0.157979,0.103479,0.144720,0.139576,0.323363
4297,You just try getting a dangerous organism past ICC quarantine. Section 22350 of the Commerce Code.,0.054277,0.058827,0.090908,0.142387,0.459514


<span style="color:blueviolet">Each document (dialog) is broken down into tokenized \"sentences\" called `Mention`. The dataframe displays confidence scores for each Mention in the dialog.</span>

In [36]:
mention_emotion_df = df[['text']].merge(emotion_df['Mention emotion'], 
                                        how='left', 
                                        left_index=True, 
                                        right_index=True).explode('Mention emotion')
mention_emotion_df[['Mention', 'Mention emotion']] = pd.DataFrame(mention_emotion_df['Mention emotion'].values.tolist(), 
                                                                  index=mention_emotion_df.index)
mention_emotion_df = mention_emotion_df.reset_index(drop=True)
mention_emotion_df = mention_emotion_df.merge(pd.DataFrame(mention_emotion_df['Mention emotion'].values.tolist()), 
                                              how='left', 
                                              left_index=True, 
                                              right_index=True).drop(['Mention emotion'], 
                                                                     axis=1).reset_index(drop=True)
mention_emotion_df

,text,Mention,anger,disgust,fear,joy,sadness
0,"No... sort of. Man, she could be cool, but all she does is get wrecked and do all the guys. She's blowin' them in the parking lot.",No...,0.168124,0.058029,0.133240,0.059511,0.531680
1,"No... sort of. Man, she could be cool, but all she does is get wrecked and do all the guys. She's blowin' them in the parking lot.",sort of.,0.060752,0.045201,0.098103,0.174564,0.390191
2,"No... sort of. Man, she could be cool, but all she does is get wrecked and do all the guys. She's blowin' them in the parking lot.","Man, she could be cool, but all she does is get wrecked and do all the guys.",0.079805,0.029354,0.184642,0.313404,0.096801
3,"No... sort of. Man, she could be cool, but all she does is get wrecked and do all the guys. She's blowin' them in the parking lot.",She's blowin' them in the parking lot.,0.057676,0.272411,0.094043,0.171805,0.105253
4,Poker sounds great. When do you play?,Poker sounds great.,0.020171,0.008876,0.025652,0.961723,0.172872
...,...,...,...,...,...,...,...
8219,You don't know. I'm sorry. I was desperate. That's not me. I shot a cop. Can you imagine what they'll do to me when I got to prison?,I shot a cop.,0.244811,0.326208,0.109765,0.048646,0.118646
8220,You don't know. I'm sorry. I was desperate. That's not me. I shot a cop. Can you imagine what they'll do to me when I got to prison?,Can you imagine what they'll do to me when I got to prison?,0.218020,0.085483,0.211432,0.026062,0.175031
8221,You just try getting a dangerous organism past ICC quarantine. Section 22350 of the Commerce Code.,You just try getting a dangerous organism past ICC quarantine.,0.064820,0.108531,0.158357,0.024554,0.291375
8222,You just try getting a dangerous organism past ICC quarantine. Section 22350 of the Commerce Code.,Section 22350 of the Commerce Code.,0.043734,0.009124,0.023458,0.260219,0.627653


## 7. Summary

<span style="color:blue">This notebook shows you how to use the Watson NLP library and how quickly and easily you can get started with Watson NLP by running the pretrained models as well as custom models for emotion analysis.